In [1]:
import getpass
import os
import json
import re
import requests

from langchain_core.documents import Document

from langchain_text_splitters import RecursiveCharacterTextSplitter

from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np

import web_scrapers
import constants
from vectorstore import vector_store

from utils import get_config

In [2]:
config = get_config()

**UPD**: Используем crawler от Apify, у него есть интеграция с Langchain. Он пробегается по сайту от главной страницы вглубь и сохраняет информацию со всех страниц.

Начинаем с главной страницы сайта, идем до глубины 3, стараемся преобразовывать html в читаемый текст, убираем порог у readableText, чтобы не пропускать даже мелкие вставки. Используем firefox браузер из playwright, чтобы читать больше информации с сайта - на нем куча javascript'а, который Crawl4AI (используемый ранее), обработать не мог.



---


**Важно:**

Для большей воспроизводимости и упрощения работы с этим блокнотом я вынес все, что связано с использованием Apify в отдельную ячейку.

Вместо ввода API-токена и ожидания краулера можно запустить соседнюю ячейку, чтобы подгрузить те же данные, посчитанные мной заранее. Они хранятся у них на сервере и не требуют API-ключа для получения.

In [ ]:
from langchain_apify import ApifyWrapper


if not os.environ.get('APIFY_API_TOKEN'):
  os.environ['APIFY_API_TOKEN'] = getpass.getpass('Enter API token for Apify: ')

apify = ApifyWrapper()

loader = apify.call_actor(
    actor_id='apify/website-content-crawler',
    run_input={
        'startUrls': [
            {'url': 'https://www.neoflex.ru/'}
            ],
        'maxCrawlPages': config.apify.max_pages,
        'maxCrawlDepth': config.apify.max_depth,
        'htmlTransformer': 'readableTextIfPossible',
        'readableTextCharThreshold': config.apify.threshold,
        'crawlerType': 'playwright:firefox'
        },
    dataset_mapping_function=lambda item: Document(
        page_content=item['text'] or '', metadata={'source': item['url']}
    ),
)

docs = loader.load()

In [3]:
# Для подгрузки без API токена
data = requests.get('https://api.apify.com/v2/datasets/oDw3TPSSsJ2dZ4ayz/items?clean=true&format=json')

docs = []
for item in data.json():
    docs.append(
        Document(
            page_content=item['text'] or '', metadata={'source': item['url']}
        )
    )

Делим полученные документы на чанки рекурсивным сплиттером:

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=config.documents.chunk_size,
    chunk_overlap=config.documents.chunk_overlap
)

all_splits = text_splitter.split_documents(docs)

Теперь займемся очищением документов от мусора.

Пишем функцию для очистки документов от навигационных артефактов, не несущих смысла:

In [5]:
def clean_navigation_artifacts(text: str):
    lines = text.splitlines()
    cleaned_lines = []
    for line in lines:
        line = line.strip()

        if not line:
            continue
        if re.fullmatch(r'20\d{2}', line):
            continue
        if re.fullmatch(r'\d{1,2}', line):
            continue

        if line.lower() in [
            'previous', 'next', 'поделиться', 'отправить на e-mail', 'узнать'
            'пресс-центр', 'новости', 'сми о нас', 'показать еще', '...',
            'подписаться на новости', 'отправить', 'поделитьсяотправить на e-mail'
        ]:
            continue

        cleaned_lines.append(line)

    return '\n'.join(cleaned_lines)

Прогоняем через нее все полученные чанки:

In [6]:
for split in all_splits:
    split.page_content = clean_navigation_artifacts(split.page_content)

Прогоняем полученные чанки через TF-IDF фильтрацию, чтобы отсеять воду и мусор. В нашем случае удалим 30% документов, худшие по TF-IDF score:

In [7]:
texts = [doc.page_content for doc in all_splits]

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

scores = tfidf_matrix.sum(axis=1)
scores = np.array(scores).flatten()

threshold = np.percentile(scores, config.tfidf.threshold_percentile)

filtered_docs = [
    doc for doc, score in zip(all_splits, scores)
    if score > threshold
]

Обогащаем полученный набор документов информацией, которую crawler от Apify не смог достать с сайта - контактами компании в разных городах, информацией о клиентах Neoflex и имейлом отдела кадров.

О контактах и клиентах компании:

- Контакты разных офисов и клиенты компании подгружаются динамически без перехода на новые страницы. Crawler не умеет работать с такими элементами
- Я пытался найти API-запрос, по которому подгружается нужная информация, чтобы обогатить документы из него, но, как оказалось, все адреса захардкожены в обычный массив внутри vue.js скрипта, а информация о клиентах лежит частично хардкодом в DOM, а частично где-то во vue.js коде
- Вместо попыток достать информацию с бэкенда я решил написать скрэппер для этих страниц, который будет прокликивать кнопки и собирать информацию вручную. Да, это долго, но это работает

Об имейле отдела кадров:

- Как и с адресами офисов, при тестах выяснилось, что нет документов о подходящем для отправки резюме имейле.
- Эта информация был отсеяна crawler'ом на этапе преобразования html в текст, так как он счел ее нерелевантной
- Также достанем ее отдельным маленьким скрэппером

Собирать информацию будем скриптами на playwright из файла web_scrapers:

Применяем скрэпперы и заливаем информацию из них в общий массив документов:

In [ ]:
city_data = await web_scrapers.scrape_city_addresses()

city_docs = [
    Document(
        metadata={'source': constants.CONTACTS_URL},
        page_content=f'Контакты офисов компании в городе {name} (адрес, электронная почта, телефон): {data}'
    )
    for name, data in city_data.items()
]

customer_data = await web_scrapers.scrape_customer_details()
customer_docs = [
    Document(
        metadata={'source': constants.CUSTOMERS_URL},
        page_content=f'Информация об одном из клиентов (заказчиков) компании Neoflex: {data}'
    )
    for data in customer_data
]

career_doc = Document(
    metadata={'source': constants.CAREER_URL},
    page_content=(await web_scrapers.scrape_career_details())
)

In [10]:
filtered_docs += city_docs
filtered_docs += customer_docs
filtered_docs.append(career_doc)

Теперь добавим теги ко всем документам для будущего поиска по ним. Ориентироваться будем на url страницы, породившей документ.

In [11]:
from constants import DOCUMENT_TAGS

for doc in filtered_docs:
    matched_tags = [
        tag for tag, snippet in DOCUMENT_TAGS.items()
        if snippet and snippet in doc.metadata.get('source', '')
    ]
    tagline = ' '.join(matched_tags) or ''
    doc.metadata['tags'] = tagline

Кроме этих тегов в теории можно добавить, например, теги городов ("Саратов", "Москва" и т.д.), сфер деятельности ("MLops", "Мобильная разработка") или компаний ("Сбер", "Россельхоз" и т.д.) парсингом page_content докуменов или прогнав их содержимое через LLM, подбирающее теги (такой подход будет использован для извлечения тегов из пользовательских запросов далее)

Наконец, добавляем обязательный префикс для E5:

In [12]:
for doc in filtered_docs:
    doc.page_content = f'passage: {doc.page_content}'

Выведем 5 рандомных чанков:

In [14]:
from random import randrange

for _ in range(5):
    idx = randrange(len(filtered_docs)-1)
    print(f'metadata: {filtered_docs[idx].metadata}')
    print(f'{filtered_docs[idx].page_content}\n\n')

metadata: {'source': 'https://www.neoflex.ru/about/customers', 'tags': 'О компании Клиенты'}
passage: Информация об одном из клиентов (заказчиков) компании Neoflex: Проекты Расчет резервов на ожидаемые кредитные убытки по МСФО (IFRS) 9 для ПАО «БАНК УРАЛСИБ» на базе системы Finastra Fusion Risk и интеграцией через Informatica PowerCenter Комплексный ИТ-аудит омниканальной микросервисной платформы Новости Банк Уралсиб и Neoflex успешно завершили аудит омниканальной платформы


metadata: {'source': 'https://www.neoflex.ru/publications/big-data-menyaet-protsess-postroeniya-otchetnosti', 'tags': ''}
passage: Полную версию публикации читайте на сайте издания NBJ.
Neoflex MSA Platform — видимая и подводная части айсберга. Статья Лины Чудновой и Юрия Корнвейца для журнала «Банковское обозрение»DevOps меняет процесс грузоперевозок. Интервью Антона Бечина для портала CNews
Контакты для МЕДИА
Если у вас есть вопросы или нужна дополнительная информация о компании, напишите нам по адресу пресс-слу

Загружаем все внутрь VectorStore:

In [15]:
document_ids = vector_store.add_documents(documents=filtered_docs)

Пишем будущий json запрос, передаем его POST запросом на сервер, получаем ответ

In [33]:
query = {
    'session_id': 'abc123',
    'question': 'В каких областях Neoflex обладает экспертизой?'
}

response = requests.post('http://127.0.0.1:8000/ask', json=query)
print(json.dumps(response.json(), indent=4, ensure_ascii=False))

{
    "answer": "Neoflex обладает экспертизой в области Site Reliability Engineering, DevOps, разработки Data-платформ, MLOps, построения аналитических платформ, Data Lake, проектирования моделей данных, построения ландшафтных архитектур, реализации трансформаций, построения BI, а также в управленческой и регуляторной отчётности.",
    "source_documents": [
        {
            "source": "https://www.neoflex.ru/expertises/sre",
            "snippet": "score: 0.192417648434639 passage: Neoflex — Экспертиза — Site Reliability Engineering\nDevOps\nSite Reliability Engineering\nРазработка Data-платформ\nТрансформация приложений, ин..."
        },
        {
            "source": "https://www.neoflex.ru/expertises/big-data",
            "snippet": "score: 0.1954480826854706 passage: Neoflex — Экспертиза — Разработка Data-платформ\nSite Reliability Engineering\nРазработка Data-платформ\nMLOps\nСоздаем платформы для обработки и..."
        }
    ],
    "session_id": "abc123"
}
